In [1]:
import pandas as pd
import numpy as np
import re
import math
import requests
import json
import os
import concurrent.futures
import warnings
warnings.filterwarnings('ignore')

#execution time for this notebook is about 70 min with a computer dealing with 12 threads, 24Go RAM and broadband access

In [2]:
pd.options.display.max_columns = None

In [3]:
dfe = pd.read_csv('../src/20221129-114802-full.csv') # file giving all the realties sold since 2017 in Lyon city (source Etalab)

In [4]:
dfe.shape

(51125, 27)

In [7]:
dfe.head()

,Unnamed: 0,index,valeur_fonciere,adresse_numero,adresse_nom_voie,nom_commune,id_parcelle,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,year,month,trim,index_prix,tuple_gps,closest_school_(m),closest_school_position,closest_bakery_(m),closest_bakery_position,closest_supermarket_(m),closest_supermarket_position,closest_metro_(m),closest_metro_position,closest_tram_(m),closest_tram_position
0,10026,24429,6196748.0,71.0,RUE MONTAGNY,Lyon 8e Arrondissement,69388000CK0097,Appartement,4.0,1.0,4.850547,45.733555,2018,10,2018-T4,3.4,"(45.733555, 4.850547)",280.70,"(45.7346644003816, 4.8537957)",277.67,"(45.7353340003816, 4.8530576)",312.26,"(45.7357634003815, 4.8530323)",1120.01,"(45.737541, 4.837294)",490.44,"(45.72953, 4.853131)"
1,14588,94779,26138036.0,7.0,RUE DE LA REPUBLIQUE,Lyon 1er Arrondissement,69381000AS0064,Appartement,20.0,1.0,4.836266,45.766464,2019,9,2019-T3,4.0,"(45.766464, 4.836266)",63.83,"(45.767023968382, 4.83644685592725)",143.52,"(45.7672747003771, 4.8348262)",119.18,"(45.7653965003773, 4.8364032)",148.20,"(45.767785, 4.836012)",971.12,"(45.75888, 4.842474)"
2,26384,187992,25025000.0,18.0,RUE DE L ARBRE SEC,Lyon 1er Arrondissement,69381000AS0078,Appartement,27.0,1.0,4.835651,45.766366,2020,12,2020-T4,6.4,"(45.766366, 4.835651)",95.73,"(45.767023968382, 4.83644685592725)",119.59,"(45.7672747003771, 4.8348262)",110.04,"(45.7664184003772, 4.8342344)",160.25,"(45.767785, 4.836012)",986.43,"(45.75888, 4.842474)"
3,17591,102152,23332406.0,4.0,RUE ETIENNE RICHERAND,Lyon 3e Arrondissement,69383000EI0081,Appartement,26.0,1.0,4.866821,45.763442,2019,4,2019-T2,4.3,"(45.763442, 4.866821)",242.84,"(45.7656024197006, 4.86636338305401)",214.96,"(45.7638234003775, 4.8641044)",37.08,"(45.7635565003776, 4.8663721)",699.31,"(45.767025, 4.859412)",384.09,"(45.764314, 4.86203)"
4,14388,94209,17473294.0,24.0,RUE NEUVE,Lyon 2e Arrondissement,69382000AC0045,Appartement,21.0,1.0,4.836262,45.765118,2019,9,2019-T3,4.0,"(45.765118, 4.836262)",97.82,"(45.7652058788412, 4.8375167949069)",241.73,"(45.7661227003772, 4.8334984)",14.10,"(45.7650821003774, 4.8360876)",194.91,"(45.763393, 4.835816)",844.60,"(45.75888, 4.842474)"


In [8]:
dfe.describe(include='all')

,Unnamed: 0,index,valeur_fonciere,adresse_numero,adresse_nom_voie,nom_commune,id_parcelle,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,year,month,trim,index_prix,tuple_gps,closest_school_(m),closest_school_position,closest_bakery_(m),closest_bakery_position,closest_supermarket_(m),closest_supermarket_position,closest_metro_(m),closest_metro_position,closest_tram_(m),closest_tram_position
count,51125.000000,51125.000000,5.112500e+04,50498.000000,51125,51125,51125,51125,51123.000000,51123.000000,51125.000000,51125.000000,51125.000000,51125.000000,51125,51125.000000,51125,51125.000000,51125,51125.000000,51125,51125.000000,51125,51125.000000,51125,51125.000000,51125
unique,NaN,NaN,NaN,NaN,1318,9,8775,2,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,8775,NaN,351,NaN,277,NaN,154,NaN,35,NaN,54
top,NaN,NaN,NaN,NaN,RUE MONTAGNY,Lyon 3e Arrondissement,69387000BN0169,Appartement,NaN,NaN,NaN,NaN,NaN,NaN,2018-T4,NaN,"(45.741026, 4.833027)",NaN,"(45.7401995329901, 4.86211831115745)",NaN,"(45.7353340003816, 4.8530576)",NaN,"(45.7564920003786, 4.7833682)",NaN,"(45.747627, 4.865189)",NaN,"(45.75888, 4.842474)"
freq,NaN,NaN,NaN,NaN,646,9569,290,49871,NaN,NaN,NaN,NaN,NaN,NaN,3508,NaN,290,NaN,1045,NaN,892,NaN,1407,NaN,4036,NaN,6679
mean,25804.657252,187864.369252,7.557937e+05,60.261991,NaN,NaN,NaN,NaN,64.173425,2.685895,4.841147,45.756441,2019.283032,6.930836,NaN,4.695550,NaN,181.970455,NaN,187.807918,NaN,293.577484,NaN,625.666203,NaN,1126.620342,NaN
std,14849.031458,134867.258226,1.958354e+06,266.292755,NaN,NaN,NaN,NaN,34.420167,1.329517,0.024602,0.015916,1.468394,3.387325,NaN,1.126399,NaN,111.134704,NaN,138.104765,NaN,239.230000,NaN,484.099999,NaN,1146.027729,NaN
min,0.000000,1.000000,1.000000e+00,1.000000,NaN,NaN,NaN,NaN,4.000000,0.000000,4.773162,45.722048,2017.000000,1.000000,NaN,2.300000,NaN,0.680000,NaN,1.860000,NaN,4.470000,NaN,17.690000,NaN,18.930000,NaN
25%,12969.000000,89474.000000,1.800000e+05,11.000000,NaN,NaN,NaN,NaN,41.000000,2.000000,4.827152,45.744744,2018.000000,4.000000,NaN,3.800000,NaN,103.290000,NaN,88.480000,NaN,120.000000,NaN,278.990000,NaN,312.230000,NaN
50%,25841.000000,185981.000000,2.775000e+05,28.000000,NaN,NaN,NaN,NaN,61.000000,3.000000,4.841737,45.754943,2019.000000,7.000000,NaN,4.500000,NaN,160.170000,NaN,154.810000,NaN,224.410000,NaN,472.960000,NaN,581.930000,NaN
75%,38640.000000,295351.000000,4.550000e+05,69.000000,NaN,NaN,NaN,NaN,80.000000,4.000000,4.860307,45.768652,2021.000000,10.000000,NaN,5.300000,NaN,231.020000,NaN,246.460000,NaN,389.530000,NaN,860.390000,NaN,1717.420000,NaN


In [9]:
dfe['nom_commune'].unique() # check that everything is in Lyon city

array(['Lyon 8e Arrondissement', 'Lyon 1er Arrondissement',
       'Lyon 3e Arrondissement', 'Lyon 2e Arrondissement',
       'Lyon 7e Arrondissement', 'Lyon 5e Arrondissement',
       'Lyon 9e Arrondissement', 'Lyon 6e Arrondissement',
       'Lyon 4e Arrondissement'], dtype=object)

In [10]:
# determine postal code
list_keys = ['Lyon 8e Arrondissement', 'Lyon 1er Arrondissement',
       'Lyon 3e Arrondissement', 'Lyon 2e Arrondissement',
       'Lyon 7e Arrondissement', 'Lyon 5e Arrondissement',
       'Lyon 9e Arrondissement', 'Lyon 6e Arrondissement',
       'Lyon 4e Arrondissement']
list_values = [69008, 69001, 69003, 69002, 69007, 69005, 69009, 69006,69004]
dict_code = dict(zip(list_keys, list_values))
def code_postal(c) :
    return dict_code[c]

In [11]:
dfe['code_postal'] = dfe['nom_commune'].apply(code_postal) # adding postal code to the dataset

In [12]:
dfe['adresse_numero'] = dfe['adresse_numero'].astype('Int64') # convert number in adress as integer

In [13]:
#create column 'rue' to gather adresse_numero and adresse_nom_voie (needed below)
def rue(num, nom) :
    if pd.notnull(num) and pd.notnull(nom) :
        return num + ' ' + nom
    if pd.notnull(num)==False :
        return nom
    if pd.notnull(nom)==False :
        return num

In [14]:
dfe['adresse_numero'] = dfe['adresse_numero'].astype(str)
dfe['adresse_nom_voie'] = dfe['adresse_nom_voie'].astype(str)
dfe['adresse'] = list(map(rue, dfe['adresse_numero'], dfe['adresse_nom_voie'])) # concatenate without nan

In [15]:
dfe.head()

,Unnamed: 0,index,valeur_fonciere,adresse_numero,adresse_nom_voie,nom_commune,id_parcelle,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,year,month,trim,index_prix,tuple_gps,closest_school_(m),closest_school_position,closest_bakery_(m),closest_bakery_position,closest_supermarket_(m),closest_supermarket_position,closest_metro_(m),closest_metro_position,closest_tram_(m),closest_tram_position,code_postal,adresse
0,10026,24429,6196748.0,71,RUE MONTAGNY,Lyon 8e Arrondissement,69388000CK0097,Appartement,4.0,1.0,4.850547,45.733555,2018,10,2018-T4,3.4,"(45.733555, 4.850547)",280.70,"(45.7346644003816, 4.8537957)",277.67,"(45.7353340003816, 4.8530576)",312.26,"(45.7357634003815, 4.8530323)",1120.01,"(45.737541, 4.837294)",490.44,"(45.72953, 4.853131)",69008,71 RUE MONTAGNY
1,14588,94779,26138036.0,7,RUE DE LA REPUBLIQUE,Lyon 1er Arrondissement,69381000AS0064,Appartement,20.0,1.0,4.836266,45.766464,2019,9,2019-T3,4.0,"(45.766464, 4.836266)",63.83,"(45.767023968382, 4.83644685592725)",143.52,"(45.7672747003771, 4.8348262)",119.18,"(45.7653965003773, 4.8364032)",148.20,"(45.767785, 4.836012)",971.12,"(45.75888, 4.842474)",69001,7 RUE DE LA REPUBLIQUE
2,26384,187992,25025000.0,18,RUE DE L ARBRE SEC,Lyon 1er Arrondissement,69381000AS0078,Appartement,27.0,1.0,4.835651,45.766366,2020,12,2020-T4,6.4,"(45.766366, 4.835651)",95.73,"(45.767023968382, 4.83644685592725)",119.59,"(45.7672747003771, 4.8348262)",110.04,"(45.7664184003772, 4.8342344)",160.25,"(45.767785, 4.836012)",986.43,"(45.75888, 4.842474)",69001,18 RUE DE L ARBRE SEC
3,17591,102152,23332406.0,4,RUE ETIENNE RICHERAND,Lyon 3e Arrondissement,69383000EI0081,Appartement,26.0,1.0,4.866821,45.763442,2019,4,2019-T2,4.3,"(45.763442, 4.866821)",242.84,"(45.7656024197006, 4.86636338305401)",214.96,"(45.7638234003775, 4.8641044)",37.08,"(45.7635565003776, 4.8663721)",699.31,"(45.767025, 4.859412)",384.09,"(45.764314, 4.86203)",69003,4 RUE ETIENNE RICHERAND
4,14388,94209,17473294.0,24,RUE NEUVE,Lyon 2e Arrondissement,69382000AC0045,Appartement,21.0,1.0,4.836262,45.765118,2019,9,2019-T3,4.0,"(45.765118, 4.836262)",97.82,"(45.7652058788412, 4.8375167949069)",241.73,"(45.7661227003772, 4.8334984)",14.10,"(45.7650821003774, 4.8360876)",194.91,"(45.763393, 4.835816)",844.60,"(45.75888, 4.842474)",69002,24 RUE NEUVE


In [16]:
# preparing threading to request API
cols = dfe.columns.to_list()
size = 4300     # my personal computer can treat 12 threads, 4300 is obtained by dividing 51 125 (nb of rows) by 12.
list_threads = np.array_split(dfe, len(dfe) // size)
list_df_threads = [pd.DataFrame(list_threads[i], columns = cols) for i in range(len(list_threads))]

In [17]:
#get geokeys by requesting the API with threads of size 'size' (there is one official geokey per unique adresse, from the official French government website)
def get_geokeys(item) :
    item['geokey'] = np.nan
    for i in item.index.to_list() :
        param = {'q' : item.at[i,'adresse'], 'postcode' : item.at[i,'code_postal']}
        r = requests.get('https://api-adresse.data.gouv.fr/search/', params = param)
        result = json.loads(r.content)
        try :
            item.at[i,'geokey'] = result['features'][0]['properties']['id']
        except :
            item.at[i,'geokey'] = 0

In [18]:
with concurrent.futures.ThreadPoolExecutor() as executor :
    executor.map(get_geokeys, list_df_threads)

In [19]:
# concatenate each dataframe obtained by threading

df_etalab = pd.DataFrame()
for item in list_df_threads :
    df_etalab = pd.concat([df_etalab, item])

In [20]:
df_etalab.head(50)

,Unnamed: 0,index,valeur_fonciere,adresse_numero,adresse_nom_voie,nom_commune,id_parcelle,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,year,month,trim,index_prix,tuple_gps,closest_school_(m),closest_school_position,closest_bakery_(m),closest_bakery_position,closest_supermarket_(m),closest_supermarket_position,closest_metro_(m),closest_metro_position,closest_tram_(m),closest_tram_position,code_postal,adresse,geokey
0,10026,24429,6196748.0,71,RUE MONTAGNY,Lyon 8e Arrondissement,69388000CK0097,Appartement,4.0,1.0,4.850547,45.733555,2018,10,2018-T4,3.4,"(45.733555, 4.850547)",280.70,"(45.7346644003816, 4.8537957)",277.67,"(45.7353340003816, 4.8530576)",312.26,"(45.7357634003815, 4.8530323)",1120.01,"(45.737541, 4.837294)",490.44,"(45.72953, 4.853131)",69008,71 RUE MONTAGNY,69388_4850_00071
1,14588,94779,26138036.0,7,RUE DE LA REPUBLIQUE,Lyon 1er Arrondissement,69381000AS0064,Appartement,20.0,1.0,4.836266,45.766464,2019,9,2019-T3,4.0,"(45.766464, 4.836266)",63.83,"(45.767023968382, 4.83644685592725)",143.52,"(45.7672747003771, 4.8348262)",119.18,"(45.7653965003773, 4.8364032)",148.20,"(45.767785, 4.836012)",971.12,"(45.75888, 4.842474)",69001,7 RUE DE LA REPUBLIQUE,0
2,26384,187992,25025000.0,18,RUE DE L ARBRE SEC,Lyon 1er Arrondissement,69381000AS0078,Appartement,27.0,1.0,4.835651,45.766366,2020,12,2020-T4,6.4,"(45.766366, 4.835651)",95.73,"(45.767023968382, 4.83644685592725)",119.59,"(45.7672747003771, 4.8348262)",110.04,"(45.7664184003772, 4.8342344)",160.25,"(45.767785, 4.836012)",986.43,"(45.75888, 4.842474)",69001,18 RUE DE L ARBRE SEC,69381_0425_00018
3,17591,102152,23332406.0,4,RUE ETIENNE RICHERAND,Lyon 3e Arrondissement,69383000EI0081,Appartement,26.0,1.0,4.866821,45.763442,2019,4,2019-T2,4.3,"(45.763442, 4.866821)",242.84,"(45.7656024197006, 4.86636338305401)",214.96,"(45.7638234003775, 4.8641044)",37.08,"(45.7635565003776, 4.8663721)",699.31,"(45.767025, 4.859412)",384.09,"(45.764314, 4.86203)",69003,4 RUE ETIENNE RICHERAND,69383_2595_00004
4,14388,94209,17473294.0,24,RUE NEUVE,Lyon 2e Arrondissement,69382000AC0045,Appartement,21.0,1.0,4.836262,45.765118,2019,9,2019-T3,4.0,"(45.765118, 4.836262)",97.82,"(45.7652058788412, 4.8375167949069)",241.73,"(45.7661227003772, 4.8334984)",14.10,"(45.7650821003774, 4.8360876)",194.91,"(45.763393, 4.835816)",844.60,"(45.75888, 4.842474)",69002,24 RUE NEUVE,69382_5005_00024
5,29185,194940,16089045.0,6,RUE ABRAHAM BLOCH,Lyon 7e Arrondissement,69387000BM0158,Appartement,20.0,1.0,4.841710,45.739307,2020,7,2020-T3,6.6,"(45.739307, 4.84171)",220.15,"(45.7389840808393, 4.83891126033237)",376.15,"(45.7361770555883, 4.83987124616591)",331.04,"(45.7421095003806, 4.8402707)",394.98,"(45.737541, 4.837294)",631.56,"(45.743679, 4.846905)",69007,6 RUE ABRAHAM BLOCH,69387_0035
6,14396,94225,19222288.0,12,RUE DE LA REPUBLIQUE,Lyon 2e Arrondissement,69382000AB0050,Appartement,24.0,1.0,4.835576,45.765109,2019,7,2019-T3,4.0,"(45.765109, 4.835576)",150.89,"(45.7654990924134, 4.83371292252651)",195.90,"(45.7642568003775, 4.8333657)",39.80,"(45.7650821003774, 4.8360876)",191.72,"(45.763393, 4.835816)",875.26,"(45.75888, 4.842474)",69002,12 RUE DE LA REPUBLIQUE,69382_6005_00012
7,14393,94222,19222288.0,12,RUE DE LA REPUBLIQUE,Lyon 2e Arrondissement,69382000AB0050,Appartement,24.0,1.0,4.835576,45.765109,2019,7,2019-T3,4.0,"(45.765109, 4.835576)",150.89,"(45.7654990924134, 4.83371292252651)",195.90,"(45.7642568003775, 4.8333657)",39.80,"(45.7650821003774, 4.8360876)",191.72,"(45.763393, 4.835816)",875.26,"(45.75888, 4.842474)",69002,12 RUE DE LA REPUBLIQUE,69382_6005_00012
8,14491,94511,22982264.0,10,RUE DE LA REPUBLIQUE,Lyon 1er Arrondissement,69381000AV0079,Appartement,29.0,1.0,4.835730,45.765498,2019,9,2019-T3,4.0,"(45.765498, 4.83573)",142.36,"(45.7652058788412, 4.8375167949069)",186.52,"(45.7661227003772, 4.8334984)",53.43,"(45.7653965003773, 4.8364032)",234.16,"(45.763393, 4.835816)",902.90,"(45.75888, 4.842474)",69001,10 RUE DE LA REPUBLIQUE,69381_6005_00010
9,12575,8815

In [19]:
df_etalab.to_csv('../src/df_etalab_geokey_221130.csv')